# API Deployment

The idea is to run this scraper as an API to have it nice and handy. The scraper code needs to be adapted to Flask, and that is what we are going to do in this Jupyter notebook. We will understand how it will work in a file that we will call **app.py**.

That file is a Flask app created to deploy an API program based on Python. With that being said, let's go!

# Flask & HTML

Flask is a microframework for web development that allows us to create simple web apps. In this case, it can be used to create a simple web page that will serve as a method to deploy the Python script **scraper.py** in a simple and portable way.

Besides, it's quite simple to understand and learn how to use. So in this step, we will understand how to create and deploy a web page based on Flask and using an *HTML template*.

### Steps to Create the API with Flask

1. **Install Flask**: First, ensure that Flask is installed in your environment.
   
2. **Create `app.py`**: This will be our main application file where we'll define our Flask routes and logic.

3. **Define Routes**: Set up routes in `app.py` to handle different API endpoints.

4. **HTML Template**: Use an HTML template to display the results on a web page.

By the end of this, you'll have a functional web scraper deployed as an API, accessible via a web page where users can input game names and get price comparisons. Let's get started!


In [2]:
from flask import Flask

app = Flask(__name__)

# where is going to be executes the app. A server, in this case, our local machine

@app.route('/')

def hello_world():
    return 'Hello world'

if __name__ == '__name__':
    app.run(debug =True)

Last cell code is the general syntax for a Flask App. We instantiate an app object from the Flask module. This is like the ground where we will build our wonderful app.

Now, what can we do with it? First, we need to understand a couple of key concepts.

1) **Routes**: This defines the URL that the app could handle and which Python function will execute when accessing this URL.

2) **HTTP Methods**: These protocols like GET and POST are the "common tongue" between a web browser and servers. A simple way to see it is that we use "GET" to retrieve data from a server and "POST" to send data to it.

   In Flask, the `request` object allows access to the data of the request that the user is sending. For example, through the web, it would send a GET or POST method.

3) **The Index Function**: This function will be executed when the user tries to access the root route.

With that in mind, the web page needs a form to be filled with the game that the user is interested in getting info about. Behind the scenes, the program will execute the scraper. So, our first step is receiving info from the web page.

This implies that the first method to be constructed is a POST method from the user. We can just omit the part in *scraper.py* where we create a function to transform the input from the user into a proper URL.

Now, we should start by developing a simple idea of how the webpage should look. But let's not worry about that right now.


In [4]:
# Here, we are designing allowed methods:
def index():
    if request.method == 'POST':
        # request.form: a dictionary with the info sent it by a html form
        # request.form.getlist('games': get alist of values associated with tag games
        items = request.form.getlist('games')
        # We requiere to construc the url within the user input on the webpage, list comprehension to do it
        us_urls = [f"https://psdeals.net/us-store/search?search_query={i.replace(' ', '+')}" for i in items]
        tr_urls = [f"https://psdeals.net/tr-store/search?search_query={i.replace(' ', '+')}" for i in items]
        # and we use the function that we already created
        us_games_info = fetch_game_info(us_urls, items, 'USD')
        tr_games_info = fetch_game_info(tr_urls, items, 'TRY')
        
        # this is the same idea that we already saw in scarper.py
        for game, details in tr_games_info.items():
            for detail in details:
                for us_detail in us_games_info.get(game, []):
                    if detail['Platform'] == us_detail['Platform']:
                        detail['Price (USD)'] = us_detail['Price (USD)']

        data = []
        for game, details in tr_games_info.items():
            for detail in details:
                detail['Game'] = game
                data.append(detail)
                
        df = pd.json_normalize(data)

        url = "https://www.google.com/search?q=lira+turca+a+dolar&oq=lira+turca+a+dolar&gs_lcrp=EgZjaHJvbWUqBggAEEUYOzIGCAAQRRg7MgYIARAuGEDSAQgyMzU4ajBqMagCALACAA&sourceid=chrome&ie=UTF-8"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept-Encoding': 'identity',
            'Connection': 'keep-alive'
        }
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, "lxml")
        requested_price = soup.find_all("span", class_="DFlfde SwHCTb")
        dolar_tr = [price['data-value'] for price in requested_price]
        dolar_tr = float(dolar_tr[0])
        print("A dolar is equal to {} TRY".format(dolar_tr))

        df["Price (TRY)"] = df["Price (TRY)"].str.replace("TL", "")
        df["Price (USD)"] = df["Price (USD)"].str.replace("FREE", "0")
        df["Price (TRY)"] = df["Price (TRY)"].str.replace("FREE", "0")
        df["Price (TRY)"] = df["Price (TRY)"].str.replace(",", "")
        df["Price (USD)"] = df["Price (USD)"].str.replace("$", "")

        df["Price (USD)"] = df["Price (USD)"].astype(float)
        df["Price (TRY)"] = df["Price (TRY)"].astype(float)
        df["Price (TRY - USD)"] = round(df["Price (TRY)"] * dolar_tr, 2)
        df["Difference (US tore- TR store)"] = round(df["Price (USD)"] - df["Price (TRY)"] * dolar_tr, 2)
        df = df.iloc[:, [0, 1, 3, 5, 6, 2]]
        # A final return: renders the data scraped in a html webpage
        return render_template('index.html', tables=[df.to_html(classes='data')], titles=df.columns.values)
    return render_template('index.html')



# Template

Alongside the API, a template should be generated on which the API will run to display the information in a browser.

A folder with the template for this browser is built.

Once this is created, a final file is added to allow the execution of the render.

The `Procfile` is created with this command for the console in development.

To accomplish this task, let us use ChatGPT to create a base template. The output from ChatGPT's prompt was already stored in this folder as `html_dev.md` with a short explanation of its structure.

In this part, I would like to focus on a better web page structure, but to give you a general idea, you could work with the structure of the `html_dev.md` file, which stores the concept of what we want.

But what really matters is that it's working. We could now add some styling, context, colors, and more with ChatGPT's help. But for now, we are done here.

# What's Next?

These simple steps:

1. In the dev folder, create the `app.py` file to store the Flask app.
2. Modify the HTML file and the template.
3. Add any necessary files such as YAML for configurations.

With these steps completed, you'll have a functioning Flask app with a web template ready to be styled and enhanced.

Now, we will use Render to deploy the webpage c:
